## Import libaries

In [ ]:
!pip uninstall dea_ml -y

In [ ]:
!pip install -e dea_ml

In [1]:
import os
import json
import xarray as xr
import geopandas as gpd
from odc.stats.tasks import TaskReader
from odc.stats.model import OutputProduct

/env/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


## Enter the (old) X and Y Tile ID of the tile you want to run

In [2]:
tile_x = '-0001'  #+0031,+0003 +0030,-0002  -0001,+0010
tile_y = '+0010'

### Open tiles geojson and find the new tile-id for the location entered 

In [4]:
tiles=gpd.read_file('https://github.com/digitalearthafrica/crop-mask/blob/main/testing/eastern_cropmask/data/gm_s2_2019-2019--P1Y.geojson?raw=true')
t=[]
for index, row in tiles.iterrows():
    x,y = tiles.loc[index]['title'].split(",")
    if (x==tile_x) & (y==tile_y): 
        x,y = int(x)+181, int(y)+77
        new_title = str(x)+","+str(y)
        t.append(x)
        t.append(y)
        print(new_title)

180,87


### Extract the tasks object for that tile

In [5]:
name, version = 'gm_s2_semiannual', '0-1-0'
op = OutputProduct(
            name='gm_s2_semiannual',
            version='0-1-0',
            short_name='gm_s2_semiannual',
            location=f"s3://dummy-bucket/{name}/{version}",
            properties={"odc:file_format": "GeoTIFF"},
            measurements=['red'],
        )

taskdb = TaskReader('s3://deafrica-services/crop_mask_eastern/1-0-0/gm_s2_semiannual_all.db', product=op)
task = taskdb.load_task(('2019--P1Y', t[0], t[1]))

### Optionally export tile geojson to view

In [6]:
with open('task_tile_check.geojson', 'w') as fh:
    json.dump(task.geobox.extent.to_crs('epsg:4326').json, fh, indent=2)

### Find the index of the tile we want to run

In [7]:
tile_index_to_run = []
all_tiles = list(taskdb.all_tiles)
for i, index in zip(all_tiles, range(0, len(all_tiles))):
    if (i[1]==t[0]) & (i[2]==t[1]):
        tile_index_to_run.append(index)
        print(index)

719


### Run the crop-mask algo using odc-stats

In [8]:
# os.system("cm-pred run "\
#           "s3://deafrica-data-dev-af/crop_mask_eastern/0-1-0/gm_s2_semiannual_all.db "\
#           "--config=https://raw.githubusercontent.com/digitalearthafrica/crop-mask/main/production/dea_ml/dea_ml/config/plugin_product.yaml "\
#           "--plugin-config=https://raw.githubusercontent.com/digitalearthafrica/crop-mask/main/production/dea_ml/dea_ml/config/ml_config.yaml "\
#           "--resolution=40 "\
#           "--threads=10 "\
#           "--memory-limit=90Gi "\
#           "--location=file:///home/jovyan/git/crop-mask/production/{product}/{version} "+str(tile_index_to_run[0])
#          )

### Or can run with cmd line, will see print statements

In [9]:
%%time
!cm-pred run s3://deafrica-services/crop_mask_eastern/1-0-0/gm_s2_semiannual_all.db --config=file:///home/jovyan/git/crop-mask/production/dea_ml/dea_ml/config/plugin_product_western.yaml  --plugin-config=file:///home/jovyan/git/crop-mask/production/dea_ml/dea_ml/config/ml_config_western_notebook.yaml  --resolution=10 --threads=7 --memory-limit=55Gi --location=file:///home/jovyan/git/crop-mask/production/{product}/{version} 719

/env/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(
[2021-08-19 08:33:19,230] {pred_run.py:150} INFO - Config overrides: {'filedb': 's3://deafrica-services/crop_mask_eastern/1-0-0/gm_s2_semiannual_all.db', 'threads': 7, 'memory_limit': '55Gi', 'output_location': 'file:///home/jovyan/git/crop-mask/production/{product}/{version}', 'heartbeat_filepath': '/tmp/cm-heartbeat.txt'}
[2021-08-19 08:33:19,231] {pred_run.py:167} INFO - Using this config: TaskRunnerConfig(filedb='s3://deafrica-services/crop_mask_eastern/1-0-0/gm_s2_semiannual_all.db', aws_unsigned=True, plugin='pred-gm-s2', plugin_config={'urls': {'slope': 'https://deafrica-input-datasets.s3.af-south-1.amazonaws.com/srtm_dem/srtm_africa_slope.tif', 'model': '///home/jovyan/git/crop-mask/testing/western_cropmask/results/western_m

## Plot the results

In [10]:
xr.open_rasterio('segmented.tif').max()

<xarray.DataArray ()>
array(366572, dtype=uint32)

In [ ]:
a= 'x'+str(t[0])
b='y0'+str(t[1])
prob=xr.open_rasterio('crop_mask_western/1-0-0/'+a+'/'+b+'/2019--P1Y/crop_mask_western_'+a+b+'_2019--P1Y_prob.tif')
print(prob)
prob.squeeze().plot.imshow(size=6);

In [ ]:
mask=xr.open_rasterio('crop_mask_western/1-0-0/'+a+'/'+b+'/2019--P1Y/crop_mask_western_'+a+b+'_2019--P1Y_mask.tif')#.squeeze().plot.imshow(size=6);
print(mask)
mask.squeeze().plot.imshow(size=6);

In [ ]:
mode=xr.open_rasterio('crop_mask_western/1-0-0/'+a+'/'+b+'/2019--P1Y/crop_mask_western_'+a+b+'_2019--P1Y_filtered.tif').squeeze()#.plot.imshow(size=6);
print(mode)
mode.squeeze().plot.imshow(size=6);

### Remove all files

In [3]:
!rm -r -f crop_mask_western

Sync files from AWS

In [ ]:
!aws s3 sync s3://deafrica-data-dev-af/crop_mask_western/ crop_mask_western --no-sign-request
